In [3]:
import os
import requests
from bs4 import BeautifulSoup, element as el

In [4]:
ROOT_URL = 'https://www.bangla-kobita.com/'
ROOT_DIR = 'bangla-kobita.com'
ROOT_PATH = os.path.join('../../data', ROOT_DIR)
if not os.path.exists(ROOT_PATH):
    os.mkdir(ROOT_PATH)

In [5]:
home_page_text = requests.get(ROOT_URL).text
soup = BeautifulSoup(home_page_text)

In [6]:
famous_h2 = soup.find_all('h2', text='খ্যাতিমান কবি')
assert len(famous_h2) == 1
famous_ul = famous_h2[0].parent.find_all('ul')
assert len(famous_ul) == 1
famous_links = famous_ul[0].findChildren('a')
famous_link: el.Tag
for famous_link in famous_links:
    print(famous_link.text, famous_link.get('href'))

অসীম সাহা /asimsaha/
আবিদ আনোয়ার /abid50anwar/
ঈশ্বরচন্দ্র গুপ্ত /ishwarchandragupta/
কাজী নজরুল ইসলাম /nazrulislam/
কামিনী রায় /kaminiroy/
জয় গোস্বামী /joygoswami/
জসীমউদ্দীন /jasimuddin/
জীবনানন্দ দাশ /jibanananda/
নির্মলেন্দু গুণ /nirmalendugoon/
মাইকেল মধুসূদন দত্ত /madhusudan/
মোহাম্মদ রফিকউজ্জামান /rafiquzzaman/
যতীন্দ্রমোহন বাগচী /jatindramohan/
রবীন্দ্রনাথ ঠাকুর /rabindranath/
রাধারানী দেবী /radharanidevi/
শক্তি চট্টোপাধ্যায় /shaktichattopadhyay/
শামসুর রাহমান /shamsurrahman/
সত্যেন্দ্রনাথ দত্ত /satyendranath/
সুকান্ত ভট্টাচার্য /sukanta/
সুকুমার রায় /sukumar/
সুনীল গঙ্গোপাধ্যায় /sunilgangopadhyay/
সুফিয়া কামাল /sufiakamal/
সৈয়দ শামসুল হক /shamsulhaque/
হেলাল হাফিজ /helalhafiz/


In [7]:
from urllib.parse import urljoin
import json
from time import sleep

def scrap_author(uri: str, name: str):
    content_path = os.path.join(ROOT_PATH, uri[1:] if uri.startswith('/') else uri)
    if not os.path.exists(content_path):
        os.mkdir(content_path)

    while True:
        url = urljoin(ROOT_URL, uri)
        author_page_text = requests.get(url).text
        author_soup = BeautifulSoup(author_page_text)
        poem_table = author_soup.select('div.tab-pane#poem table', attrs={'class': 'post-list'})
        assert len(poem_table) == 1, f'{len(poem_table)}: {url}'
        poem_links = poem_table[0].findChildren('a')
        poem_link: el.Tag
        for poem_link in poem_links:
            poem_uri = poem_link.get('href')
            poem_url = urljoin(ROOT_URL, poem_uri)
            poem_html = requests.get(poem_url).text
            poem_soup = BeautifulSoup(poem_html)
            h1 = poem_soup.find_all('h1')
            assert len(h1) == 1
            title = h1[0].text
            content_div = poem_soup.find_all('div', attrs={'class': 'post-content'})
            assert len(content_div) == 1
            content = content_div[0].get_text(separator='\n')

            filename = list(filter(bool, poem_uri.split('/')))[-1]
            file_path = os.path.join(content_path, f'{filename}.json')
            with open(file_path, 'w', encoding='utf-8') as content_file:
                data_dict = {'title': title.strip(), 'author': name.strip(), 'url': poem_url, 'content': content.strip()}
                json.dump(data_dict, content_file, ensure_ascii=False)

        next_page_li = author_soup.find_all('li', attrs={'class': 'PagedList-skipToNext'})
        assert len(next_page_li) <= 1, len(next_page_li)
        if len(next_page_li) == 0:
            break

        uri = next_page_li[0].find('a').get('href')

for famous_link in famous_links:
    scrap_author(uri=famous_link.get('href'), name=famous_link.text)
    print(f'Completed: {famous_link.text}')
    sleep(5)

Completed: অসীম সাহা
Completed: আবিদ আনোয়ার
Completed: ঈশ্বরচন্দ্র গুপ্ত
Completed: কাজী নজরুল ইসলাম
Completed: কামিনী রায়
Completed: জয় গোস্বামী
Completed: জসীমউদ্দীন
Completed: জীবনানন্দ দাশ
Completed: নির্মলেন্দু গুণ
Completed: মাইকেল মধুসূদন দত্ত
Completed: মোহাম্মদ রফিকউজ্জামান
Completed: যতীন্দ্রমোহন বাগচী
Completed: রবীন্দ্রনাথ ঠাকুর
Completed: রাধারানী দেবী
Completed: শক্তি চট্টোপাধ্যায়
Completed: শামসুর রাহমান
Completed: সত্যেন্দ্রনাথ দত্ত
Completed: সুকান্ত ভট্টাচার্য
Completed: সুকুমার রায়
Completed: সুনীল গঙ্গোপাধ্যায়
Completed: সুফিয়া কামাল
Completed: সৈয়দ শামসুল হক
Completed: হেলাল হাফিজ


In [8]:
print(f'Total {sum(len(files) for *_, files in os.walk(ROOT_PATH))} entries scrapped')

Total 4472 entries scrapped
